In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('/Users/saikiranpatnana/Documents/DiamondPricePrediction_1/notebooks/data/gemstone.csv')

In [6]:
df

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [8]:
df.drop(labels=['id'],axis=1,inplace=True)

In [9]:
X = df.drop('price',axis=1)

In [14]:
# y = df['price'] will create a array of the price;

In [15]:
y = df[['price']] #it will create a dataframe with price

In [16]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [26]:
cat_cols = X.columns[X.dtypes==object]
num_cols = X.columns[X.dtypes!=object]

In [27]:
cat_cols = ['cut', 'color', 'clarity']

Index(['cut', 'color', 'clarity'], dtype='object')

In [28]:
num_cols = ['carat', 'depth', 'table', 'x', 'y', 'z']

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [29]:
# aliter:
# cat_cols = x.select_dtypes(include=object).columns
# num_cols = x.select_dtypes(exclude=object).columns

In [34]:
X.cut.unique()

array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object)

In [33]:
(X.cut.value_counts())

cut
Ideal        92454
Premium      49910
Very Good    37566
Good         11622
Fair          2021
Name: count, dtype: int64

In [35]:
cut_cats = [ 'Fair','Good', 'Very Good', 'Premium',  'Ideal']
color_cats = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_cats = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [37]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [79]:
# num_pipeline = Pipeline(steps=[('std_scaler',StandardScaler()),('imputer',SimpleImputer(strategy='median'))])
# cat_pipeline = Pipeline(steps=[('std_scaler',StandardScaler()),
# ('ordinal_encoder',OrdinalEncoder(categories=[cut_cats,color_cats,clarity_cats])),('imputer',SimpleImputer(strategy='most_frequent'))])
# preprocessor = ColumnTransformer([('cat_pipeline',cat_pipeline,cat_cols),('num_pipeline',num_pipeline,num_cols)])

In [83]:

num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('std_scaler',StandardScaler())

    ]

)

cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_cats,color_cats,clarity_cats])),
    ('std_scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,num_cols),
('cat_pipeline',cat_pipeline,cat_cols)
])


In [84]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state = 32)

In [85]:
x_train = pd.DataFrame(preprocessor.fit_transform(x_train), columns=preprocessor.get_feature_names_out())
x_test = pd.DataFrame(preprocessor.transform(x_test), columns=preprocessor.get_feature_names_out())

In [86]:
x_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.888807,-0.851198,1.441566,-0.944040,-0.927206,-0.993419,-0.129364,1.529933,0.685799
1,0.473470,1.181202,-1.163589,0.589495,0.616053,0.703190,-2.137944,-0.935041,0.019230
2,-0.607702,-0.574052,-0.642558,-0.502021,-0.491462,-0.529389,0.874927,0.297446,1.352368
3,2.852048,0.072620,-1.163589,2.321487,2.277325,2.283791,0.874927,2.146176,0.019230
4,0.041001,-1.497871,1.441566,0.318871,0.271089,0.166655,-0.129364,-0.318798,0.685799
...,...,...,...,...,...,...,...,...,...
135496,2.635814,-1.682634,0.399504,2.177155,2.150233,1.950270,-0.129364,2.146176,-1.313909
135497,-0.845560,0.719293,-0.121527,-0.962081,-0.927206,-0.877412,0.874927,0.297446,0.019230
135498,-0.888807,-0.574052,-0.642558,-0.907957,-0.927206,-0.949916,-1.133654,0.297446,0.019230
135499,0.473470,1.365966,1.441566,0.589495,0.543429,0.703190,-0.129364,-0.935041,-0.647340


In [49]:
from sklearn.linear_model import  LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import numpy as np

In [99]:
models_to_be_trained = {'linear_regressor':LinearRegression(),'lasso_regressor':Lasso(),'ridge_regressor':Ridge(),'elastic_net':ElasticNet()}
def evaluate_model(y_test,y_pred) :
    mse = mean_squared_error(y_test,y_pred)
    mae = mean_absolute_error(y_test,y_pred)
    rmse = np.sqrt(mse)
    r_square = r2_score(y_test,y_pred)
    return mse,mae,rmse,r_square
def train_model(model,x_train,y_train,x_test):
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    model_intercept = model.intercept_
    model_coef = model.coef_
    return y_pred,model_intercept,model_coef
for i in range(len(list(models_to_be_trained))):
    model = list(models_to_be_trained.values())[i]
    y_pred,model_intercept,model_coef = train_model(model,x_train,y_train,x_test)
    mse,mae,rmse,r_square = evaluate_model(y_test,y_pred)
    print('model: ',list(models_to_be_trained.keys())[i])
    print('model_performance_metrics: ')
    print('model_intercept: ',model_intercept)
    # print('model_coeff: ', model_coef)
    print('mse: ', mse)
    print('mae: ', mae)
    print('rmse: ', rmse)
    print('r2_score: ', r_square)
    print('*'*40)
    print('\n')




model:  linear_regressor
model_performance_metrics: 
model_intercept:  [3970.08498092]
mse:  1021471.2631449125
mae:  677.2021094891585
rmse:  1010.6786151615718
r2_score:  0.9371701066078356
****************************************


model:  lasso_regressor
model_performance_metrics: 
model_intercept:  [3970.08498092]
mse:  1021929.2436779374
mae:  678.40351049175
rmse:  1010.9051605753813
r2_score:  0.9371419365857274
****************************************


model:  ridge_regressor
model_performance_metrics: 
model_intercept:  [3970.08498092]
mse:  1021459.947502919
mae:  677.2308920416291
rmse:  1010.673017104404
r2_score:  0.937170802624068
****************************************


model:  elastic_net
model_performance_metrics: 
model_intercept:  [3970.08498092]
mse:  2328870.3754430083
mae:  1058.8702566849374
rmse:  1526.0636865619365
r2_score:  0.8567530162691461
****************************************


